In [ ]:
%%bash
mkdir -p /content/data

wget -O /content/data/results.csv \
  https://raw.githubusercontent.com/martj42/international_results/master/results.csv

wget -O /content/data/fifa_rankings.csv \
  https://raw.githubusercontent.com/Dato-Futbol/fifa-ranking/master/ranking_fifa_historical.csv

wget -O /content/data/wc_matches.csv \
  https://raw.githubusercontent.com/jfjelstul/worldcup/master/data-csv/matches.csv

ls -lh /content/data


total 6.4M
-rw-r--r-- 1 root root 2.6M Jan 19 16:32 fifa_rankings.csv
-rw-r--r-- 1 root root 3.6M Jan 19 16:32 results.csv
-rw-r--r-- 1 root root 292K Jan 19 16:32 wc_matches.csv


--2026-01-19 16:32:25--  https://raw.githubusercontent.com/martj42/international_results/master/results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3686208 (3.5M) [text/plain]
Saving to: ‘/content/data/results.csv’

     0K .......... .......... .......... .......... ..........  1% 23.6M 0s
    50K .......... .......... .......... .......... ..........  2% 32.6M 0s
   100K .......... .......... .......... .......... ..........  4%  124M 0s
   150K .......... .......... .......... .......... ..........  5%  152M 0s
   200K .......... .......... .......... .......... ..........  6% 47.8M 0s
   250K .......... .......... .......... .......... ..........  8% 87.4M 0s
   300K .......... .......... .......... .......... ..........  9%  108M 0s
   350K ....

In [ ]:
import pandas as pd
import numpy as np

results = pd.read_csv("/content/data/results.csv")
fifa = pd.read_csv("/content/data/fifa_rankings.csv")
wc_matches = pd.read_csv("/content/data/wc_matches.csv")

results["date"] = pd.to_datetime(results["date"], errors="coerce")
results = results.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

# Win/Draw/Loss label (Option 1: ignore shootouts)
results["FTR"] = np.where(results["home_score"] > results["away_score"], "H",
                   np.where(results["home_score"] < results["away_score"], "A", "D"))

print(results.shape, fifa.shape, wc_matches.shape)
results.head()


(48943, 10) (67894, 6) (1248, 37)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,FTR
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,D
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,H
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,H
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,D
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,H


In [ ]:
print(fifa.columns)
print(fifa.head(5))

print("Date range:", results["date"].min().date(), "to", results["date"].max().date())
print("Total matches:", len(results))



Index(['team', 'total_points', 'date', 'id', 'id_num', 'team_short'], dtype='object')
        team  total_points        date       id  id_num team_short
0  Argentina       1855.20  2024-02-15  id14289   14289        ARG
1     France       1845.44  2024-02-15  id14289   14289        FRA
2    England       1800.05  2024-02-15  id14289   14289        ENG
3    Belgium       1798.46  2024-02-15  id14289   14289        BEL
4     Brazil       1784.09  2024-02-15  id14289   14289        BRA
Date range: 1872-11-30 to 2026-01-18
Total matches: 48943


In [ ]:
import pandas as pd
import numpy as np

# Clean FIFA table
fifa = fifa.copy()
fifa["date"] = pd.to_datetime(fifa["date"], errors="coerce")
fifa = fifa.dropna(subset=["date"])
fifa = fifa.sort_values(["team", "date"]).reset_index(drop=True)

# keep only needed cols
fifa = fifa[["team", "date", "total_points"]].copy()

# --- Clean results table (if not already done) ---
results = results.copy()
results["date"] = pd.to_datetime(results["date"], errors="coerce")
results = results.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

# --- merge_asof helper (as-of match date, per team) ---
def merge_fifa_points(df, team_col: str, prefix: str) -> pd.DataFrame:
    tmp = fifa.rename(columns={"team": team_col, "date": "fifa_date"})
    out = pd.merge_asof(
        df.sort_values("date"),
        tmp.sort_values("fifa_date"),
        left_on="date",
        right_on="fifa_date",
        by=team_col,
        direction="backward"
    )
    out = out.rename(columns={"total_points": f"{prefix}_fifa_points"})
    return out.drop(columns=["fifa_date"])

# FIFA points for both teams
data = results.copy()
data = merge_fifa_points(data, "home_team", "home")
data = merge_fifa_points(data, "away_team", "away")

# If a match is before 1992 (no FIFA points), fill with 0 (or you can drop them)
data["home_fifa_points"] = data["home_fifa_points"].fillna(0)
data["away_fifa_points"] = data["away_fifa_points"].fillna(0)

# Strength difference feature
data["fifa_points_diff"] = data["home_fifa_points"] - data["away_fifa_points"]

min_fifa_date = fifa["date"].min()
data_after = data[data["date"] >= min_fifa_date]
data_after[["date","home_team","away_team","home_fifa_points","away_fifa_points","fifa_points_diff"]].head(10)

#data[["date","home_team","away_team","home_fifa_points","away_fifa_points","fifa_points_diff"]].head(10)


,date,home_team,away_team,home_fifa_points,away_fifa_points,fifa_points_diff
18703,1993-01-01,Ghana,Mali,34.0,22.0,12.0
18704,1993-01-02,Gabon,Burkina Faso,27.0,11.0,16.0
18705,1993-01-02,Kuwait,Lebanon,21.0,0.0,21.0
18706,1993-01-03,Burkina Faso,Mali,11.0,22.0,-11.0
18707,1993-01-03,Gabon,Ghana,27.0,34.0,-7.0
18708,1993-01-08,Uganda,Tanzania,12.0,15.0,-3.0
18709,1993-01-10,Angola,Zimbabwe,10.0,27.0,-17.0
18710,1993-01-10,Botswana,South Africa,2.0,5.0,-3.0
18711,1993-01-10,DR Congo,Cameroon,0.0,43.0,-43.0
18712,1993-01-10,Senegal,Algeria,27.0,39.0,-12.0


In [ ]:
import pandas as pd
import numpy as np

# Ensure sorted by date
data = data.sort_values("date").reset_index(drop=True).copy()

# Create an ID for each match so we can merge features back
data["match_id"] = np.arange(len(data))

# Home perspective
home_log = data[["match_id","date","home_team","away_team","home_score","away_score","neutral","tournament"]].copy()
home_log.rename(columns={
    "home_team":"team",
    "away_team":"opponent",
    "home_score":"gf",
    "away_score":"ga"
}, inplace=True)
home_log["is_home"] = 1

# Away perspective
away_log = data[["match_id","date","away_team","home_team","away_score","home_score","neutral","tournament"]].copy()
away_log.rename(columns={
    "away_team":"team",
    "home_team":"opponent",
    "away_score":"gf",
    "home_score":"ga"
}, inplace=True)
away_log["is_home"] = 0

team_log = pd.concat([home_log, away_log], ignore_index=True)
team_log = team_log.sort_values(["team","date","match_id"]).reset_index(drop=True)

# Match outcome from the TEAM perspective: W/D/L
team_log["points"] = np.where(team_log["gf"] > team_log["ga"], 3,
                       np.where(team_log["gf"] < team_log["ga"], 0, 1))
team_log["win"]  = (team_log["points"] == 3).astype(int)
team_log["draw"] = (team_log["points"] == 1).astype(int)
team_log["loss"] = (team_log["points"] == 0).astype(int)

team_log.head()


,match_id,date,team,opponent,gf,ga,neutral,tournament,is_home,points,win,draw,loss
0,36241,2012-09-25,Abkhazia,Artsakh,1,1,False,Friendly,1,1,0,1,0
1,36384,2012-10-21,Abkhazia,Artsakh,0,3,False,Friendly,0,0,0,0,1
2,37742,2014-06-01,Abkhazia,Occitania,1,1,True,CONIFA World Football Cup,1,1,0,1,0
3,37756,2014-06-02,Abkhazia,Sápmi,2,1,False,CONIFA World Football Cup,0,3,1,0,0
4,37777,2014-06-04,Abkhazia,South Ossetia,0,0,True,CONIFA World Football Cup,1,1,0,1,0


In [ ]:
def add_form_features(team_log: pd.DataFrame, windows=(5,10)) -> pd.DataFrame:
    df = team_log.copy()
    df = df.sort_values(["team","date","match_id"]).reset_index(drop=True)

    base_cols = ["gf","ga","points","win","draw","loss"]
    for w in windows:
        for c in base_cols:
            df[f"{c}_roll{w}"] = (
                df.groupby("team")[c]
                  .transform(lambda s: s.shift(1).rolling(w, min_periods=w).mean())
            )
    return df

team_log_feat = add_form_features(team_log, windows=(5,10))

# Keep only rows where form is available (both windows ready)
need = [c for c in team_log_feat.columns if c.endswith("roll5") or c.endswith("roll10")]
team_log_feat = team_log_feat.dropna(subset=need).reset_index(drop=True)

team_log_feat[["team","date","gf_roll5","ga_roll5","points_roll5","win_roll5","gf_roll10","ga_roll10"]].head()


,team,date,gf_roll5,ga_roll5,points_roll5,win_roll5,gf_roll10,ga_roll10
0,Abkhazia,2016-05-29,1.0,0.8,1.6,0.4,0.9,1.0
1,Abkhazia,2016-05-31,2.2,0.2,2.0,0.6,1.7,0.9
2,Abkhazia,2016-06-01,2.4,0.0,2.6,0.8,1.8,0.6
3,Abkhazia,2016-06-04,2.6,0.0,2.6,0.8,1.9,0.5
4,Abkhazia,2016-06-05,2.8,0.0,2.6,0.8,1.9,0.4


In [ ]:
# Split home-side rows and away-side rows, then merge back to match_id
home_form = team_log_feat[team_log_feat["is_home"] == 1].copy()
away_form = team_log_feat[team_log_feat["is_home"] == 0].copy()

form_cols = [c for c in team_log_feat.columns if "roll" in c]

home_form = home_form[["match_id"] + form_cols].copy()
home_form.columns = ["match_id"] + [f"home_{c}" for c in form_cols]

away_form = away_form[["match_id"] + form_cols].copy()
away_form.columns = ["match_id"] + [f"away_{c}" for c in form_cols]

# Merge into match table
model_df = data.merge(home_form, on="match_id", how="inner").merge(away_form, on="match_id", how="inner")

# Label H/D/A already exists in results as FTR; ensure it's there
model_df["FTR"] = np.where(model_df["home_score"] > model_df["away_score"], "H",
                    np.where(model_df["home_score"] < model_df["away_score"], "A", "D"))

# Quick check
print("Before form merge:", data.shape)
print("After form merge:", model_df.shape)

model_df[["date","home_team","away_team","FTR","home_points_roll5","away_points_roll5","fifa_points_diff"]].head()


Before form merge: (48943, 14)
After form merge: (46547, 38)


,date,home_team,away_team,FTR,home_points_roll5,away_points_roll5,fifa_points_diff
0,1881-03-12,England,Scotland,A,1.8,2.4,0.0
1,1882-03-11,Scotland,England,H,3.0,1.2,0.0
2,1882-03-13,Wales,England,H,1.2,1.2,0.0
3,1882-03-25,Scotland,Wales,H,3.0,1.8,0.0
4,1883-02-03,England,Wales,H,0.6,1.8,0.0


In [ ]:
import pandas as pd
import numpy as np

# We use the same match history used for form (data has match_id, date, teams, scores)
h2h_base = data[["match_id","date","home_team","away_team","home_score","away_score"]].copy()
h2h_base = h2h_base.sort_values(["date","match_id"]).reset_index(drop=True)

# Team A = home
h2h_home = h2h_base.copy()
h2h_home["team"] = h2h_home["home_team"]
h2h_home["opponent"] = h2h_home["away_team"]
h2h_home["gf"] = h2h_home["home_score"]
h2h_home["ga"] = h2h_home["away_score"]

# Team B = away
h2h_away = h2h_base.copy()
h2h_away["team"] = h2h_away["away_team"]
h2h_away["opponent"] = h2h_away["home_team"]
h2h_away["gf"] = h2h_away["away_score"]
h2h_away["ga"] = h2h_away["home_score"]

h2h_log = pd.concat([h2h_home, h2h_away], ignore_index=True)
h2h_log = h2h_log[["match_id","date","team","opponent","gf","ga"]].sort_values(["team","opponent","date","match_id"]).reset_index(drop=True)

# Team-perspective outcome in this specific head-to-head match
h2h_log["points"] = np.where(h2h_log["gf"] > h2h_log["ga"], 3,
                     np.where(h2h_log["gf"] < h2h_log["ga"], 0, 1))
h2h_log["win"]  = (h2h_log["points"] == 3).astype(int)
h2h_log["draw"] = (h2h_log["points"] == 1).astype(int)
h2h_log["loss"] = (h2h_log["points"] == 0).astype(int)

h2h_log.head()


,match_id,date,team,opponent,gf,ga,points,win,draw,loss
0,36241,2012-09-25,Abkhazia,Artsakh,1,1,1,0,1,0
1,36384,2012-10-21,Abkhazia,Artsakh,0,3,0,0,0,1
2,42429,2019-06-04,Abkhazia,Artsakh,1,1,1,0,1,0
3,39650,2016-05-29,Abkhazia,Chagos Islands,9,0,3,1,0,0
4,42414,2019-06-02,Abkhazia,Chameria,3,1,3,1,0,0


In [ ]:
def add_h2h_features(df: pd.DataFrame, window=5) -> pd.DataFrame:
    df = df.sort_values(["team","opponent","date","match_id"]).reset_index(drop=True).copy()
    g = df.groupby(["team","opponent"], sort=False)

    # how many meetings happened BEFORE this match
    df["h2h_matches_prior"] = g.cumcount()

    # all-time prior averages
    df["h2h_points_avg_prior"] = g["points"].transform(lambda s: s.shift(1).expanding().mean())
    df["h2h_win_rate_prior"]   = g["win"].transform(lambda s: s.shift(1).expanding().mean())
    df["h2h_draw_rate_prior"]  = g["draw"].transform(lambda s: s.shift(1).expanding().mean())
    df["h2h_loss_rate_prior"]  = g["loss"].transform(lambda s: s.shift(1).expanding().mean())

    # last-5 meetings prior (rolling)
    df[f"h2h_points_roll{window}"] = g["points"].transform(lambda s: s.shift(1).rolling(window, min_periods=window).mean())
    df[f"h2h_win_roll{window}"]    = g["win"].transform(lambda s: s.shift(1).rolling(window, min_periods=window).mean())
    df[f"h2h_draw_roll{window}"]   = g["draw"].transform(lambda s: s.shift(1).rolling(window, min_periods=window).mean())
    df[f"h2h_loss_roll{window}"]   = g["loss"].transform(lambda s: s.shift(1).rolling(window, min_periods=window).mean())

    return df

h2h_feat = add_h2h_features(h2h_log, window=5)

# Keep only useful columns
h2h_cols = [
    "h2h_matches_prior",
    "h2h_points_avg_prior","h2h_win_rate_prior","h2h_draw_rate_prior","h2h_loss_rate_prior",
    "h2h_points_roll5","h2h_win_roll5","h2h_draw_roll5","h2h_loss_roll5"
]
h2h_feat = h2h_feat[["match_id","team","opponent"] + h2h_cols].copy()

h2h_feat.head()


,match_id,team,opponent,h2h_matches_prior,h2h_points_avg_prior,h2h_win_rate_prior,h2h_draw_rate_prior,h2h_loss_rate_prior,h2h_points_roll5,h2h_win_roll5,h2h_draw_roll5,h2h_loss_roll5
0,36241,Abkhazia,Artsakh,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36384,Abkhazia,Artsakh,1,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
2,42429,Abkhazia,Artsakh,2,0.5,0.0,0.5,0.5,NaN,NaN,NaN,NaN
3,39650,Abkhazia,Chagos Islands,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42414,Abkhazia,Chameria,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create home-side H2H features
home_h2h = h2h_feat.rename(columns={c: f"home_{c}" for c in h2h_cols})
home_h2h = home_h2h.rename(columns={"team":"home_team", "opponent":"away_team"})
home_h2h = home_h2h[["match_id","home_team","away_team"] + [f"home_{c}" for c in h2h_cols]]

# Create away-side H2H features
away_h2h = h2h_feat.rename(columns={c: f"away_{c}" for c in h2h_cols})
away_h2h = away_h2h.rename(columns={"team":"away_team", "opponent":"home_team"})
away_h2h = away_h2h[["match_id","home_team","away_team"] + [f"away_{c}" for c in h2h_cols]]

# Merge into model_df
model_df2 = model_df.merge(home_h2h, on=["match_id","home_team","away_team"], how="left") \
                    .merge(away_h2h, on=["match_id","home_team","away_team"], how="left")

print("Before H2H merge:", model_df.shape)
print("After H2H merge:", model_df2.shape)

model_df2[[
    "date","home_team","away_team","FTR",
    "home_h2h_matches_prior","away_h2h_matches_prior",
    "home_h2h_points_avg_prior","away_h2h_points_avg_prior"
]].head(10)


Before H2H merge: (46547, 38)
After H2H merge: (46547, 56)


,date,home_team,away_team,FTR,home_h2h_matches_prior,away_h2h_matches_prior,home_h2h_points_avg_prior,away_h2h_points_avg_prior
0,1881-03-12,England,Scotland,A,9,9,0.888889,1.888889
1,1882-03-11,Scotland,England,H,10,10,2.000000,0.800000
2,1882-03-13,Wales,England,H,3,3,1.000000,2.000000
3,1882-03-25,Scotland,Wales,H,6,6,3.000000,0.000000
4,1883-02-03,England,Wales,H,4,4,1.500000,1.500000
5,1883-03-10,England,Scotland,A,11,11,0.727273,2.090909
6,1883-03-12,Wales,Scotland,A,7,7,0.000000,3.000000
7,1884-03-15,Scotland,England,H,12,12,2.166667,0.666667
8,1884-03-17,Wales,England,A,5,5,1.200000,1.800000
9,1884-03-29,Scotland,Wales,H,8,8,3.000000,0.000000


In [ ]:
# Choose required columns (form + basic h2h roll5)
required = [
    "home_points_roll5","away_points_roll5",
    "home_points_roll10","away_points_roll10",
    "home_h2h_points_roll5","away_h2h_points_roll5"
]

train_df = model_df2.dropna(subset=required).reset_index(drop=True)

print("Trainable matches:", train_df.shape)
train_df[required].head()


Trainable matches: (25395, 56)


,home_points_roll5,away_points_roll5,home_points_roll10,away_points_roll10,home_h2h_points_roll5,away_h2h_points_roll5
0,1.8,2.4,1.0,2.7,0.6,2.4
1,3.0,1.2,2.7,1.2,2.4,0.6
2,3.0,1.8,2.7,0.9,3.0,0.0
3,1.8,3.0,1.5,2.7,0.6,2.4
4,1.2,3.0,0.9,2.7,0.0,3.0


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Remove duplicate columns (safety)
train_df = train_df.loc[:, ~train_df.columns.duplicated()].copy()

y = train_df["FTR"]

# Choose numeric features in a controlled way (no double-adding)
num_features = [
    "home_fifa_points","away_fifa_points","fifa_points_diff",
    # Form (roll5/roll10)
    "home_gf_roll5","home_ga_roll5","home_points_roll5","home_win_roll5","home_draw_roll5","home_loss_roll5",
    "home_gf_roll10","home_ga_roll10","home_points_roll10","home_win_roll10","home_draw_roll10","home_loss_roll10",
    "away_gf_roll5","away_ga_roll5","away_points_roll5","away_win_roll5","away_draw_roll5","away_loss_roll5",
    "away_gf_roll10","away_ga_roll10","away_points_roll10","away_win_roll10","away_draw_roll10","away_loss_roll10",
    # H2H (prior + roll5)
    "home_h2h_matches_prior","home_h2h_points_avg_prior","home_h2h_win_rate_prior","home_h2h_draw_rate_prior","home_h2h_loss_rate_prior",
    "home_h2h_points_roll5","home_h2h_win_roll5","home_h2h_draw_roll5","home_h2h_loss_roll5",
    "away_h2h_matches_prior","away_h2h_points_avg_prior","away_h2h_win_rate_prior","away_h2h_draw_rate_prior","away_h2h_loss_rate_prior",
    "away_h2h_points_roll5","away_h2h_win_roll5","away_h2h_draw_roll5","away_h2h_loss_roll5",
]

# Keep only columns that exist (in case some are missing)
num_features = [c for c in num_features if c in train_df.columns]

cat_features = ["home_team","away_team","tournament","neutral"]
cat_features = [c for c in cat_features if c in train_df.columns]

X = train_df[num_features + cat_features].copy()

preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ],
    remainder="drop"
)

pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=3000, multi_class="multinomial"))
])

# Time-based split
train_size = int(len(train_df) * 0.80)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("Confusion matrix:\n", confusion_matrix(y_test, pred, labels=["H","D","A"]))
print("\nReport:\n", classification_report(y_test, pred))


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.5516834022445364
Confusion matrix:
 [[1840  100  442]
 [ 700   94  488]
 [ 465   82  868]]

Report:
               precision    recall  f1-score   support

           A       0.48      0.61      0.54      1415
           D       0.34      0.07      0.12      1282
           H       0.61      0.77      0.68      2382

    accuracy                           0.55      5079
   macro avg       0.48      0.49      0.45      5079
weighted avg       0.51      0.55      0.50      5079



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
proba = pipe.predict_proba(X_test)
classes = pipe.named_steps["clf"].classes_

out = train_df.iloc[train_size:][["date","home_team","away_team","FTR"]].copy()
for i, cls in enumerate(classes):
    out[f"P({cls})"] = proba[:, i]

out["Pred"] = pipe.predict(X_test)
out.head(15)


,date,home_team,away_team,FTR,P(A),P(D),P(H),Pred
20316,2016-10-08,Solomon Islands,New Caledonia,A,0.328305,0.248482,0.423213,H
20317,2016-10-08,Mexico,New Zealand,H,0.202407,0.191796,0.605797,H
20318,2016-10-08,DR Congo,Libya,H,0.207218,0.236499,0.556283,H
20319,2016-10-08,Germany,Czech Republic,H,0.095313,0.205970,0.698717,H
20320,2016-10-08,Senegal,Cape Verde,H,0.042100,0.154882,0.803018,H
20321,2016-10-08,Belize,Honduras,A,0.514076,0.331097,0.154827,A
20322,2016-10-08,Ivory Coast,Mali,H,0.093225,0.176640,0.730135,H
20323,2016-10-08,Scotland,Lithuania,D,0.138348,0.142178,0.719474,H
20324,2016-10-09,Tunisia,Guinea,H,0.097967,0.271060,0.630973,H
20325,2016-10-09,Zambia,Nigeria,A,0.287653,0.468211,0.244137,D


In [ ]:
import numpy as np
import pandas as pd

# ---------- helpers ----------
def _fifa_points_asof(team: str, match_date: pd.Timestamp, fifa_df: pd.DataFrame) -> float:
    sub = fifa_df[(fifa_df["team"] == team) & (fifa_df["date"] <= match_date)]
    if len(sub) == 0:
        return 0.0
    return float(sub.iloc[-1]["total_points"])

def _team_past_matches(team: str, match_date: pd.Timestamp, results_df: pd.DataFrame) -> pd.DataFrame:
    df = results_df[results_df["date"] < match_date].copy()
    home = df[df["home_team"] == team][["date","home_team","away_team","home_score","away_score"]].copy()
    home["gf"] = home["home_score"]
    home["ga"] = home["away_score"]

    away = df[df["away_team"] == team][["date","away_team","home_team","away_score","home_score"]].copy()
    away.columns = ["date","home_team","away_team","home_score","away_score"]
    away["gf"] = away["home_score"]
    away["ga"] = away["away_score"]

    out = pd.concat([home, away], ignore_index=True)
    out = out.sort_values("date").reset_index(drop=True)

    out["points"] = np.where(out["gf"] > out["ga"], 3, np.where(out["gf"] < out["ga"], 0, 1))
    out["win"]  = (out["points"] == 3).astype(int)
    out["draw"] = (out["points"] == 1).astype(int)
    out["loss"] = (out["points"] == 0).astype(int)
    return out

def _form_features(team: str, match_date: pd.Timestamp, results_df: pd.DataFrame, windows=(5,10)) -> dict:
    past = _team_past_matches(team, match_date, results_df)
    feats = {}

    for w in windows:
        last = past.tail(w)
        if len(last) < w:
            # not enough history -> NaN (you can change to 0.0 if you prefer)
            feats[f"gf_roll{w}"] = np.nan
            feats[f"ga_roll{w}"] = np.nan
            feats[f"points_roll{w}"] = np.nan
            feats[f"win_roll{w}"] = np.nan
            feats[f"draw_roll{w}"] = np.nan
            feats[f"loss_roll{w}"] = np.nan
        else:
            feats[f"gf_roll{w}"] = last["gf"].mean()
            feats[f"ga_roll{w}"] = last["ga"].mean()
            feats[f"points_roll{w}"] = last["points"].mean()
            feats[f"win_roll{w}"] = last["win"].mean()
            feats[f"draw_roll{w}"] = last["draw"].mean()
            feats[f"loss_roll{w}"] = last["loss"].mean()
    return feats

def _h2h_features(team: str, opp: str, match_date: pd.Timestamp, results_df: pd.DataFrame, window=5) -> dict:
    df = results_df[results_df["date"] < match_date].copy()

    # matches where these two teams played each other (any order)
    m = df[((df["home_team"] == team) & (df["away_team"] == opp)) |
           ((df["home_team"] == opp) & (df["away_team"] == team))].copy()

    m = m.sort_values("date").reset_index(drop=True)

    feats = {}
    feats["h2h_matches_prior"] = int(len(m))

    if len(m) == 0:
        # no history -> NaN
        feats.update({
            "h2h_points_avg_prior": np.nan,
            "h2h_win_rate_prior": np.nan,
            "h2h_draw_rate_prior": np.nan,
            "h2h_loss_rate_prior": np.nan,
            f"h2h_points_roll{window}": np.nan,
            f"h2h_win_roll{window}": np.nan,
            f"h2h_draw_roll{window}": np.nan,
            f"h2h_loss_roll{window}": np.nan,
        })
        return feats

    # compute TEAM-perspective outcomes for each H2H game
    # if team was home
    m_team_home = m[(m["home_team"] == team)].copy()
    m_team_home["gf"] = m_team_home["home_score"]
    m_team_home["ga"] = m_team_home["away_score"]

    # if team was away
    m_team_away = m[(m["away_team"] == team)].copy()
    m_team_away["gf"] = m_team_away["away_score"]
    m_team_away["ga"] = m_team_away["home_score"]

    t = pd.concat([m_team_home[["date","gf","ga"]], m_team_away[["date","gf","ga"]]], ignore_index=True)
    t = t.sort_values("date").reset_index(drop=True)

    t["points"] = np.where(t["gf"] > t["ga"], 3, np.where(t["gf"] < t["ga"], 0, 1))
    t["win"]  = (t["points"] == 3).astype(int)
    t["draw"] = (t["points"] == 1).astype(int)
    t["loss"] = (t["points"] == 0).astype(int)

    feats["h2h_points_avg_prior"] = t["points"].mean()
    feats["h2h_win_rate_prior"]   = t["win"].mean()
    feats["h2h_draw_rate_prior"]  = t["draw"].mean()
    feats["h2h_loss_rate_prior"]  = t["loss"].mean()

    last = t.tail(window)
    if len(last) < window:
        feats[f"h2h_points_roll{window}"] = np.nan
        feats[f"h2h_win_roll{window}"]    = np.nan
        feats[f"h2h_draw_roll{window}"]   = np.nan
        feats[f"h2h_loss_roll{window}"]   = np.nan
    else:
        feats[f"h2h_points_roll{window}"] = last["points"].mean()
        feats[f"h2h_win_roll{window}"]    = last["win"].mean()
        feats[f"h2h_draw_roll{window}"]   = last["draw"].mean()
        feats[f"h2h_loss_roll{window}"]   = last["loss"].mean()

    return feats


# ---------- main function ----------
def predict_match(teamA: str, teamB: str, date: str, tournament="Friendly", neutral=True):
    """
    Predict match result for TeamA vs TeamB (TeamA is treated as home_team).
    Returns probabilities for H/D/A and the final prediction.
    """
    match_date = pd.to_datetime(date)

    # FIFA features
    home_fp = _fifa_points_asof(teamA, match_date, fifa)
    away_fp = _fifa_points_asof(teamB, match_date, fifa)

    row = {
        "date": match_date,
        "home_team": teamA,
        "away_team": teamB,
        "tournament": tournament,
        "neutral": bool(neutral),

        "home_fifa_points": home_fp,
        "away_fifa_points": away_fp,
        "fifa_points_diff": home_fp - away_fp,
    }

    # Form features (home/away sides)
    home_form = _form_features(teamA, match_date, results, windows=(5,10))
    away_form = _form_features(teamB, match_date, results, windows=(5,10))

    for k, v in home_form.items():
        row[f"home_{k}"] = v
    for k, v in away_form.items():
        row[f"away_{k}"] = v

    # H2H features (teamA vs teamB, and teamB vs teamA)
    home_h2h = _h2h_features(teamA, teamB, match_date, results, window=5)
    away_h2h = _h2h_features(teamB, teamA, match_date, results, window=5)

    for k, v in home_h2h.items():
        row[f"home_{k}"] = v
    for k, v in away_h2h.items():
        row[f"away_{k}"] = v

    # Build DataFrame for the pipeline (must include the same columns used in training)
    X_one = pd.DataFrame([row])

    # Predict
    proba = pipe.predict_proba(X_one)[0]
    classes = pipe.named_steps["clf"].classes_
    pred = pipe.predict(X_one)[0]

    probs = dict(zip(classes, proba))
    return {
        "match": f"{teamA} vs {teamB}",
        "date": str(match_date.date()),
        "tournament": tournament,
        "neutral": bool(neutral),
        "probabilities": probs,  # keys: 'H', 'D', 'A'
        "prediction": pred
    }


In [ ]:
predict_match("England", "France", "2022-12-10", tournament="FIFA World Cup", neutral=True)


{'match': 'England vs France',
 'date': '2022-12-10',
 'tournament': 'FIFA World Cup',
 'neutral': True,
 'probabilities': {'A': np.float64(0.6084161978618611),
  'D': np.float64(0.1064736197431773),
  'H': np.float64(0.2851101823949617)},
 'prediction': 'A'}

In [ ]:
def predict_match_pretty_percent(teamA: str, teamB: str, date: str, tournament="Friendly", neutral=True):
    res = predict_match(teamA, teamB, date, tournament=tournament, neutral=neutral)

    probs = res["probabilities"]  # {'H':..., 'D':..., 'A':...}

    # TeamA=home, TeamB=away
    p_home = float(probs.get("H", 0.0)) * 100
    p_draw = float(probs.get("D", 0.0)) * 100
    p_away = float(probs.get("A", 0.0)) * 100

    pred_code = res["prediction"]
    if pred_code == "H":
        pred_text = f"{teamA} WIN"
    elif pred_code == "A":
        pred_text = f"{teamB} WIN"
    else:
        pred_text = "DRAW"

    print("="*55)
    print(f"Match: {teamA} vs {teamB}")
    print(f"Date: {res['date']} | Tournament: {tournament} | Neutral: {neutral}")
    print("-"*55)
    print(f"{teamA} win: {p_home:.1f}%")
    print(f"Draw:        {p_draw:.1f}%")
    print(f"{teamB} win: {p_away:.1f}%")
    print("-"*55)
    print(f"✅ Model prediction: {pred_text}")
    print("="*55)

    return {
        "match": f"{teamA} vs {teamB}",
        "date": res["date"],
        "tournament": tournament,
        "neutral": bool(neutral),
        f"{teamA}_win_%": round(p_home, 2),
        "draw_%": round(p_draw, 2),
        f"{teamB}_win_%": round(p_away, 2),
        "prediction": pred_text
    }


In [ ]:
predict_match_pretty_percent("England", "France", "2022-12-10", tournament="FIFA World Cup", neutral=True)


Match: England vs France
Date: 2022-12-10 | Tournament: FIFA World Cup | Neutral: True
-------------------------------------------------------
England win: 28.5%
Draw:        10.6%
France win: 60.8%
-------------------------------------------------------
✅ Model prediction: France WIN


{'match': 'England vs France',
 'date': '2022-12-10',
 'tournament': 'FIFA World Cup',
 'neutral': True,
 'England_win_%': 28.51,
 'draw_%': 10.65,
 'France_win_%': 60.84,
 'prediction': 'France WIN'}

In [ ]:
predict_match("England", "France", "2022-12-10", tournament="FIFA World Cup", neutral=True)

{'match': 'England vs France',
 'date': '2022-12-10',
 'tournament': 'FIFA World Cup',
 'neutral': True,
 'probabilities': {'A': np.float64(0.6084161978618611),
  'D': np.float64(0.1064736197431773),
  'H': np.float64(0.2851101823949617)},
 'prediction': 'A'}

In [ ]:
predict_match_pretty_percent("England", "France", "2026-06-10", tournament="FIFA World Cup", neutral=True)


Match: England vs France
Date: 2026-06-10 | Tournament: FIFA World Cup | Neutral: True
-------------------------------------------------------
England win: 31.0%
Draw:        13.7%
France win: 55.3%
-------------------------------------------------------
✅ Model prediction: France WIN


{'match': 'England vs France',
 'date': '2026-06-10',
 'tournament': 'FIFA World Cup',
 'neutral': True,
 'England_win_%': 31.04,
 'draw_%': 13.65,
 'France_win_%': 55.3,
 'prediction': 'France WIN'}

In [ ]:
import pandas as pd
import numpy as np

def h2h_summary(teamA: str, teamB: str, date: str, results_df: pd.DataFrame):
    d = pd.to_datetime(date)

    df = results_df[results_df["date"] < d].copy()

    # all past matches between the two (either order)
    m = df[((df["home_team"] == teamA) & (df["away_team"] == teamB)) |
           ((df["home_team"] == teamB) & (df["away_team"] == teamA))].copy()

    m = m.sort_values("date").reset_index(drop=True)

    total = len(m)
    if total == 0:
        return {
            "h2h_matches": 0,
            f"{teamA}_wins": 0,
            "draws": 0,
            f"{teamB}_wins": 0,
        }

    # Determine winner per match (team names)
    def winner(row):
        if row["home_score"] > row["away_score"]:
            return row["home_team"]
        if row["home_score"] < row["away_score"]:
            return row["away_team"]
        return "Draw"

    m["winner"] = m.apply(winner, axis=1)

    a_wins = int((m["winner"] == teamA).sum())
    b_wins = int((m["winner"] == teamB).sum())
    draws  = int((m["winner"] == "Draw").sum())

    return {
        "h2h_matches": total,
        f"{teamA}_wins": a_wins,
        "draws": draws,
        f"{teamB}_wins": b_wins
    }


In [ ]:
h2h_summary("England", "France", "2026-06-10", results)


{'h2h_matches': 34, 'England_wins': 17, 'draws': 6, 'France_wins': 11}

In [ ]:
import pandas as pd
import numpy as np

def last5_h2h(teamA: str, teamB: str, date: str, results_df: pd.DataFrame):
    d = pd.to_datetime(date)

    df = results_df[results_df["date"] < d].copy()

    # filter matches where these two teams played (either order)
    m = df[((df["home_team"] == teamA) & (df["away_team"] == teamB)) |
           ((df["home_team"] == teamB) & (df["away_team"] == teamA))].copy()

    if m.empty:
        return pd.DataFrame(columns=["date","home_team","away_team","home_score","away_score","winner"])

    m = m.sort_values("date").reset_index(drop=True)

    def winner(row):
        if row["home_score"] > row["away_score"]:
            return row["home_team"]
        elif row["home_score"] < row["away_score"]:
            return row["away_team"]
        return "Draw"

    m["winner"] = m.apply(winner, axis=1)

    # take last 5 meetings and show most recent first
    last5 = m.tail(5).iloc[::-1].copy()

    return last5[["date","home_team","away_team","home_score","away_score","winner"]]


In [ ]:
last5_h2h("Argentina", "France", "2026-06-10", results)


,date,home_team,away_team,home_score,away_score,winner
12,2022-12-18,Argentina,France,3,3,Draw
11,2018-06-30,France,Argentina,4,3,France
10,2009-02-11,France,Argentina,0,2,Argentina
9,2007-02-07,France,Argentina,0,1,Argentina
8,1986-03-26,France,Argentina,2,0,France


In [ ]:
last5_h2h("Bangladesh", "India", "2026-06-10", results)

,date,home_team,away_team,home_score,away_score,winner
28,2025-11-18,Bangladesh,India,1,0,Bangladesh
27,2025-03-25,India,Bangladesh,0,0,Draw
26,2021-10-04,Bangladesh,India,1,1,Draw
25,2021-06-07,Bangladesh,India,0,2,India
24,2019-10-15,India,Bangladesh,1,1,Draw


In [ ]:
def predict_match_pretty_percent_with_h2h(teamA: str, teamB: str, date: str, tournament=None, neutral=True):
    # handle null/empty tournament
    if tournament is None or str(tournament).strip() == "":
        tournament = "Friendly"   # default choice

    # prediction
    res = predict_match(teamA, teamB, date, tournament=tournament, neutral=neutral)
    probs = res["probabilities"]

    p_home = float(probs.get("H", 0.0)) * 100
    p_draw = float(probs.get("D", 0.0)) * 100
    p_away = float(probs.get("A", 0.0)) * 100

    pred_code = res["prediction"]
    if pred_code == "H":
        pred_text = f"{teamA} WIN"
    elif pred_code == "A":
        pred_text = f"{teamB} WIN"
    else:
        pred_text = "DRAW"

    # h2h stats (before the match date)
    h2h = h2h_summary(teamA, teamB, date, results)

    # print nicely
    print("="*60)
    print(f"Match: {teamA} vs {teamB}")
    print(f"Date: {res['date']} | Tournament: {tournament} | Neutral: {neutral}")
    print("-"*60)
    print(f"{teamA} win: {p_home:.2f}%")
    print(f"Draw:        {p_draw:.2f}%")
    print(f"{teamB} win: {p_away:.2f}%")
    print("-"*60)
    print(f"✅ Model prediction: {pred_text}")
    print("-"*60)
    print("Head-to-Head (before this date):")
    print(f"Total meetings: {h2h['h2h_matches']}")
    print(f"{teamA} wins:    {h2h.get(f'{teamA}_wins', 0)}")
    print(f"Draws:         {h2h.get('draws', 0)}")
    print(f"{teamB} wins:    {h2h.get(f'{teamB}_wins', 0)}")
    print("="*60)

    return {
        "match": f"{teamA} vs {teamB}",
        "date": res["date"],
        "tournament": tournament,
        "neutral": bool(neutral),
        f"{teamA}_win_%": round(p_home, 2),
        "draw_%": round(p_draw, 2),
        f"{teamB}_win_%": round(p_away, 2),
        "prediction": pred_text,
        "h2h": h2h
    }


In [ ]:
predict_match_pretty_percent_with_h2h("Bangladesh", "India", "2026-01-15", tournament=None, neutral=True)


Match: Bangladesh vs India
Date: 2026-01-15 | Tournament: Friendly | Neutral: True
------------------------------------------------------------
Bangladesh win: 23.12%
Draw:        30.71%
India win: 46.17%
------------------------------------------------------------
✅ Model prediction: India WIN
------------------------------------------------------------
Head-to-Head (before this date):
Total meetings: 29
Bangladesh wins:    4
Draws:         13
India wins:    12


{'match': 'Bangladesh vs India',
 'date': '2026-01-15',
 'tournament': 'Friendly',
 'neutral': True,
 'Bangladesh_win_%': 23.12,
 'draw_%': 30.71,
 'India_win_%': 46.17,
 'prediction': 'India WIN',
 'h2h': {'h2h_matches': 29,
  'Bangladesh_wins': 4,
  'draws': 13,
  'India_wins': 12}}

In [ ]:
def last_n_matches(team: str, n: int = 5, date: str | None = None, results_df: pd.DataFrame = None):
    """
    Show last n matches for a team.
    - If date is given: shows matches before that date
    - If date is None: shows the latest n matches available in the dataset
    """
    if results_df is None:
        results_df = results

    # if no date is given, set match_date to "after the last match" so it includes all history
    if date is None:
        match_date = results_df["date"].max() + pd.Timedelta(days=1)
    else:
        match_date = pd.to_datetime(date)

    past = _team_past_matches(team, match_date, results_df).copy()

    if past.empty:
        return f"No matches found for {team} before {match_date.date()}."

    # Make readable columns
    past["opponent"] = np.where(past["home_team"] == team, past["away_team"], past["home_team"])
    past["score"] = past["gf"].astype(int).astype(str) + "-" + past["ga"].astype(int).astype(str)

    past["result"] = np.where(past["gf"] > past["ga"], "W",
                      np.where(past["gf"] < past["ga"], "L", "D"))

    cols = ["date", "opponent", "score", "result", "points"]
    return past.tail(n)[cols].reset_index(drop=True)


In [ ]:
last_n_matches("Bangladesh", n=10, date="2026-06-01")


,date,opponent,score,result,points
0,2024-11-13,Maldives,0-1,L,0
1,2024-11-16,Maldives,2-1,W,3
2,2025-03-25,India,0-0,D,1
3,2025-06-04,Bhutan,2-0,W,3
4,2025-06-10,Singapore,1-2,L,0
5,2025-09-06,Nepal,0-0,D,1
6,2025-10-09,Hong Kong,3-4,L,0
7,2025-10-14,Hong Kong,1-1,D,1
8,2025-11-13,Nepal,2-2,D,1
9,2025-11-18,India,1-0,W,3


In [ ]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,FTR
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,D
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,H
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,H
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,D
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,H
